<a href="https://colab.research.google.com/github/raushan9jnv/M.techThesisProject/blob/main/ncert_mtech_dec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
# for github push inalid notebook error
from google.colab import _message
_message.blocking_request('clean_metadata', {})


{'error': {'type': 'NO_HANDLER_ERROR',
  'description': 'No handler was provided for the given request type: clean_metadata'}}

In [1]:
%%capture
# 1. Update pip to ensure it can find the latest wheels
!pip install --upgrade pip

# 2. Install Unsloth first
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# 3. Install dependencies WITHOUT version constraints on xformers
# We let the system find the binary that matches the installed PyTorch
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

## We will use Qwen 2.5 (7B) Instruct. It is currently the SOTA open-weights model for multilingual tasks (including Hindi).

In [11]:
from unsloth import FastLanguageModel
import torch

# 1. Configuration
max_seq_length = 2048 # Supports RoPE Scaling internally, so choose any!
dtype = None # None for auto detection. Float16 for Tesla T4, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 2. Load the Model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-7B-Instruct", # You can change this to "Qwen/Qwen2.5-1.5B-Instruct" for faster training
    # model_name = "Qwen/Qwen2.5-1.5B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

print("Model loaded successfully!")

==((====))==  Unsloth 2025.12.5: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.16G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Model loaded successfully!


## Configure LoRA (Low-Rank Adaptation)
This is the "Research Level" part. We target specific modules to ensure the model learns the Hindi structures and logic, not just memorizes words.

In [12]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,  # Rank Stabilized LoRA
    loftq_config = None, # And LoftQ
)

## Cell 4: Create & Load Dataset
I have included a script here to create a dummy train.jsonl file based on the data you provided. If you have your own file, upload it to Colab and skip the creation part.

In [16]:
import json
from datasets import Dataset

# --- STEP A: Create the dummy file (Skip if you uploaded 'train.jsonl') ---
# data = [
#     {"messages": [{"role": "system", "content": "You are a Hindi NCERT tutor for Class 1, Subject Hindi."}, {"role": "user", "content": "Context: अम्मा, आज लगा दे झूला! इस झूले पर मैं झूलूँगा। इस पर चढ़कर, ऊपर बढ़कर, आसमान को मैं छू लूँगा। झूला झूल रही है डाली, झूल रहा है पत्ता-पत्ता। इस बड़े झूले पर झूलकर बच्चा बहुत खुश है।\nQuestion: बच्चा अपनी अम्मा से क्या लगाने के लिए कह रहा है?"}, {"role": "assistant", "content": "बच्चा अपनी अम्मा से एक झूला लगाने के लिए कह रहा है। वह झूले पर झूलना चाहता है और झूलते-झूलते आसमान को छूना चाहता है, जिससे उसे बहुत खुशी मिलेगी।"}]},
#     {"messages": [{"role": "system", "content": "You are a Hindi NCERT tutor for Class 1, Subject Hindi."}, {"role": "user", "content": "Context: एक लड़का पेड़ पर आम देखता है। वह गुलेल से आम पर निशाना लगाता है। आम नीचे गिर जाता है। एक कौआ उस आम को उठा लेता है और उड़ जाता है। रास्ते में एक लोमड़ी कौए को देखती है।\nQuestion: लड़के ने आम को पेड़ से तोड़ने के लिए क्या इस्तेमाल किया?"}, {"role": "assistant", "content": "लड़के ने आम को पेड़ से तोड़ने के लिए गुलेल का इस्तेमाल किया। उसने गुलेल से आम पर निशाना लगाया ताकि वह नीचे गिर जाए।"}]},
#     {"messages": [{"role": "system", "content": "You are a Hindi NCERT tutor for Class 1, Subject Hindi."}, {"role": "user", "content": "Context: छह साल की छोकरी भरकर लाई टोकरी। टोकरी में आम हैं, नहीं बताती दाम है। दिखा-दिखाकर टोकरी, बुलाती है छोकरी। हमें नहीं बुलाती है, नाम नहीं अब पूछना। नाम उसका है कुछ और, नाम है उसका छबीली।\nQuestion: टोकरी में क्या भरा हुआ है?"}, {"role": "assistant", "content": "टोकरी में आम भरे हुए हैं। कविता में साफ-साफ बताया गया है कि 'टोकरी में आम हैं' और वह छोकरी आम बेचने के लिए ही टोकरी लेकर आई है।"}]}
# ]

# # Write to a file
# with open("train.jsonl", "w", encoding="utf-8") as f:
#     for entry in data:
#         json.dump(entry, f, ensure_ascii=False)
#         f.write("\n")

# --- STEP B: Load and Format Data ---
from unsloth.chat_templates import get_chat_template

# Setup correct chat template for Qwen
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, etc.
    mapping = {"role" : "role", "content" : "content", "user" : "user", "assistant" : "assistant"}, # ShareGPT style
)

def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

# Load Dataset
dataset = Dataset.from_json("train.jsonl")
dataset = dataset.map(formatting_prompts_func, batched = True,)

print("Dataset prepared. Sample entry:")
print(dataset[0]["text"])
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/3591 [00:00<?, ? examples/s]

Dataset prepared. Sample entry:
<|im_start|>system
You are a Hindi NCERT tutor for Class 1, Subject Hindi. Topic: स्वर.<|im_end|>
<|im_start|>user
Context: हिंदी भाषा में कुछ अक्षर ऐसे होते हैं जिन्हें बोलने के लिए किसी और अक्षर की मदद नहीं लेनी पड़ती। इन्हें स्वर कहते हैं। अ, आ, इ, ई, उ, ऊ, ऋ, ए, ऐ, ओ, औ, अं, अः ये सभी स्वर हैं। स्वरों की अपनी मात्राएँ होती हैं।

Question: स्वर किसे कहते हैं और हिंदी भाषा में कितने मुख्य स्वर होते हैं?<|im_end|>
<|im_start|>assistant
स्वर वे अक्षर होते हैं जिन्हें बोलने के लिए किसी अन्य अक्षर की सहायता नहीं लेनी पड़ती। ये स्वतंत्र रूप से बोले जाते हैं। हिंदी भाषा में मुख्य रूप से 11 स्वर होते हैं: अ, आ, इ, ई, उ, ऊ, ऋ, ए, ऐ, ओ, औ। अं और अः को अयोगवाह कहते हैं।<|im_end|>

Dataset({
    features: ['messages', 'text'],
    num_rows: 3591
})


## Cell 5: Train the Model
This creates the Trainer. We use SFTTrainer (Supervised Fine-Tuning).

In [17]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # Set this to roughly (num_epochs * num_rows) / batch_size. For your full data, try 1 epoch first.
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer_stats = trainer.train()

Map (num_proc=2):   0%|          | 0/3591 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,591 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 40,370,176 of 7,655,986,688 (0.53% trained)


Step,Training Loss
1,1.088600
2,1.019700
3,1.020400
4,1.130200
5,1.058600
6,1.212000
7,0.910400
8,1.095900
9,1.186800
10,0.936200


### Cell 6: Inference (Test the Model)
Let's see if it learned to act like the NCERT tutor.

In [20]:
# Enable native 2x faster inference
FastLanguageModel.for_inference(model)

messages = [
    {"role": "system", "content": "You are a Hindi NCERT tutor for Class 1, Subject Hindi."},
    # This is a NEW question not in the training set, but similar style
    # {"role": "user", "content": "Context: एक दिन एक लड़की थी। उसने आम के पेड़ पर एक बड़ा आम देखा।\nQuestion: लड़की ने पेड़ पर क्या देखा?"},
    {"role": "user", "content": "Question: लड़की ने पेड़ पर क्या देखा? and from where u getting this answer"},
    # {"role": "user", "content": "Question: कुंई बनाने वाले 'चेजारो' की क्या विशेषताएँ होती हैं? tpoic name"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must be True for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 128, use_cache = True, temperature = 0.1)
decoded_output = tokenizer.batch_decode(outputs)

# Clean up the output to just show the assistant response
response = decoded_output[0].split("<|im_start|>assistant\n")[-1].replace("<|im_end|>", "")
print("User Question:", messages[1]['content'])
print("-" * 30)
print("Model Response:", response)

User Question: Question: लड़की ने पेड़ पर क्या देखा? and from where u getting this answer
------------------------------
Model Response: इस प्रश्न का उत्तर आपको एक संदर्भ या एक वाक्य में मिल सकता है जहाँ लड़की ने पेड़ पर कुछ देखा है। अगर आपको इसका संदर्भ नहीं मिल रहा है, तो �


### Evaluation

In [7]:
!pip install rouge_score sacrebleu evaluate pandas

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24987 sha256=e3d873d5fb14ce441660987dfd3cd0359157232d190eba02660f829655c60dd1
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [evaluate]


In [21]:
import torch
from unsloth import FastLanguageModel
import pandas as pd
from tqdm import tqdm
from evaluate import load

# 1. SETUP: Prepare Metrics
rouge = load("rouge")
bleu = load("bleu")

# 2. DATA: The Test Set (Questions the model has NEVER seen)
test_data = [
    {
        "question": "भोजन के मुख्य स्रोत क्या हैं?",
        "reference": "भोजन के मुख्य स्रोत पौधे और जानवर होते हैं। पौधों से हमें अनाज मिलता है और जानवरों से दूध और मांस।"
    },
    {
        "question": "कुंई बनाने वाले 'चेजारो' की क्या विशेषताएँ होती हैं?",
        "reference": "चेजारो वे कुशल कारीगर होते हैं जो राजस्थान में कुंई बनाने का काम करते हैं। वे संकरी जगह में खुदाई करने में माहिर होते हैं।"
    },
    {
        "question": "सूर्य क्या है?",
        "reference": "सूर्य एक तारा है जो सौरमंडल के केंद्र में स्थित है। यह हमारी पृथ्वी के लिए ऊर्जा का मुख्य स्रोत है।"
    },
    {
        "question": "लोकतंत्र (Democracy) का क्या अर्थ है?",
        "reference": "लोकतंत्र का अर्थ है 'लोगों का शासन'। यह ऐसी व्यवस्था है जहाँ जनता अपना शासक खुद चुनती है।"
    },
    {
        "question": "फसल किसे कहते हैं?",
        "reference": "जब एक ही किस्म के पौधे किसी स्थान पर बड़े पैमाने पर उगाए जाते हैं, तो इसे फसल कहते हैं। जैसे गेहूँ की फसल।"
    }
]

# 3. INFERENCE LOOP
FastLanguageModel.for_inference(model) # Optimize for speed

results = []

print("Running Research Evaluation (Base vs. Fine-Tuned)...")

for item in tqdm(test_data):
    # A. Prepare Input
    messages = [
        {"role": "system", "content": "You are a Hindi NCERT tutor."},
        {"role": "user", "content": f"Question: {item['question']}"}
    ]
    inputs = tokenizer.apply_chat_template(
        messages, tokenize=True, add_generation_prompt=True, return_tensors="pt"
    ).to("cuda")

    # --- MODE 1: Fine-Tuned Model (The "After" State) ---
    # We run generation normally. The LoRA adapters are ACTIVE.
    outputs_ft = model.generate(input_ids=inputs, max_new_tokens=128, use_cache=True, temperature=0.1)
    ans_ft = tokenizer.batch_decode(outputs_ft)[0].split("<|im_start|>assistant\n")[-1].replace("<|im_end|>", "").strip()

    # --- MODE 2: Original Base Model (The "Before" State) ---
    # We strictly DISABLE the adapters to see how raw Qwen performs.
    with model.disable_adapter():
        outputs_base = model.generate(input_ids=inputs, max_new_tokens=128, use_cache=True, temperature=0.1)
        ans_base = tokenizer.batch_decode(outputs_base)[0].split("<|im_start|>assistant\n")[-1].replace("<|im_end|>", "").strip()

    # Save Results
    results.append({
        "Question": item['question'],
        "Reference Answer": item['reference'],
        "Base Model (Original)": ans_base,
        "NCERT-Qwen (Fine-Tuned)": ans_ft
    })

# 4. SCORING & DATAFRAME
df = pd.DataFrame(results)

# Calculate Scores for Fine-Tuned Model
rouge_score = rouge.compute(predictions=df["NCERT-Qwen (Fine-Tuned)"], references=df["Reference Answer"])
bleu_score = bleu.compute(predictions=df["NCERT-Qwen (Fine-Tuned)"], references=df["Reference Answer"])

# 5. DISPLAY RESULTS
print("\n" + "="*60)
print(" RESEARCH RESULTS FOR THESIS (CHAPTER 5)")
print("="*60)
print(f"ROUGE-L Score: {rouge_score['rougeL']:.4f}")
print(f"BLEU Score:    {bleu_score['bleu']:.4f}")
print("-" * 60)

# Show the comparison table
pd.set_option('display.max_colwidth', None) # Show full text
display(df)

# 6. EXPORT
df.to_csv("thesis_comparison_results.csv", index=False)
print("\n✅ Saved 'thesis_comparison_results.csv'. Copy these rows to your Qualitative Analysis section!")

Running Research Evaluation (Base vs. Fine-Tuned)...


100%|██████████| 5/5 [01:18<00:00, 15.62s/it]


 RESEARCH RESULTS FOR THESIS (CHAPTER 5)
ROUGE-L Score: 0.0000
BLEU Score:    0.0290
------------------------------------------------------------


,Question,Reference Answer,Base Model (Original),NCERT-Qwen (Fine-Tuned)
0,भोजन के मुख्य स्रोत क्या हैं?,भोजन के मुख्य स्रोत पौधे और जानवर होते हैं। पौधों से हमें अनाज मिलता है और जानवरों से दूध और मांस।,"भोजन के मुख्य स्रोत निम्नलिखित हैं:\n\n1. ग्रामीण उत्पाद: \n - शमार (चावल, जूट, चना)\n - फल और फलियाँ\n - अम्लीय और निष्क्रिय फल\n - शैवाल और अ","भोजन के मुख्य स्रोत निम्नलिखित हैं:\n\n1. ग्रास-सेवक जैसे चावल, बाजरा, गेहूँ, दाल, चना, राजमा, फल और शैवाल\n\n2. अपोटोफोग्लोप्टिक जैसे �"
1,कुंई बनाने वाले 'चेजारो' की क्या विशेषताएँ होती हैं?,चेजारो वे कुशल कारीगर होते हैं जो राजस्थान में कुंई बनाने का काम करते हैं। वे संकरी जगह में खुदाई करने में माहिर होते हैं।,चेजारो (Turmeric) की कुंई बनाने वाली विशेषताएँ इस प्रकार होती हैं:\n\n1. रंग: चेजारो का रंग गहरा नारंगी होता है।\n\n2. मूल्यवान रूप: यह एक मूल्यवान,चेजारो (कुंई बनाने वाले पेड़) की कुछ महत्वपूर्ण विशेषताएँ इस प्रकार समझाई जा सकती हैं:\n\n1. फूल: चेजारो के फूल अपने आप में बहुत दिलचस्प हो
2,सूर्य क्या है?,सूर्य एक तारा है जो सौरमंडल के केंद्र में स्थित है। यह हमारी पृथ्वी के लिए ऊर्जा का मुख्य स्रोत है।,"सूर्य एक सूर्यास्त्र, यानी ग्रहण शरद्वतीय दिव्य तेजस्वी प्रकाश विकारण वाला ग्रह है। यह हमारे ग्रहण शरद्वती में सबसे बRIGHT और महत्वपूर्ण अवस",सूर्य एक स्वप्रकाशी ग्रह है जो हमारे अपने सौर सिद्धांत में केन्द्रीय ग्रह है। इसके बारे में कुछ महत्वपूर्ण तथ्य हैं:\n\n1. सूर्य हमारे सौर प्रणाल
3,लोकतंत्र (Democracy) का क्या अर्थ है?,लोकतंत्र का अर्थ है 'लोगों का शासन'। यह ऐसी व्यवस्था है जहाँ जनता अपना शासक खुद चुनती है।,लोकतंत्र (Democracy) का अर्थ निम्नलिखित है:\n\n1. सभी नागरिकों के लिए समान अधिकार और जनता की शक्ति: लोकतंत्र में सभी नागरिकों को समान अधिकार और जनता की �,लोकतंत्र (Democracy) एक सामाजिक और सांस्कृतिक संरचना है जिसमें निर्वाचित संसद की शक्ति और आपसी बहुमत के आधार पर निर्णय लेने का अधिकार है। इसका मु
4,फसल किसे कहते हैं?,"जब एक ही किस्म के पौधे किसी स्थान पर बड़े पैमाने पर उगाए जाते हैं, तो इसे फसल कहते हैं। जैसे गेहूँ की फसल।",फसल किसे कहते हैं?\n\nफसल कोई सालाना या वर्षावार उत्पादन है जो खेत में अवशोषण करके पौधों द्वारा बनाया जाता है। फसल आमतौर पर ग्रामीण आर्थिक �,"फसल कोई उत्पादन है जो वनस्पति से बनाई जाती है। विशेष रूप से, फसल उत्पादन का एक प्रकार है जो आदमी द्वारा उत्पादित किया जाता है और इसका म"



✅ Saved 'thesis_comparison_results.csv'. Copy these rows to your Qualitative Analysis section!
